In [1]:
import pandas as pd
import py_stringmatching as sm

In [2]:
sunshine = pd.read_csv('../Data/Outputs_Cleanup/physicians_info_dedup.csv',dtype=str)

In [3]:
part_d = pd.read_csv('../Data/Outputs_Cleanup/prescriber_dedup.csv',dtype=str)

In [4]:
part_d.head()

,id,fname,lname,type,city
0,1447344023,Aleksandr,Itkin,Dermatology,San Diego
1,1184722209,David,Hughes,Dentist,Milpitas
2,1164561353,Pamela,Knickerbocker,Family Practice,Shasta Lake
3,1225025422,A,Jaffer,Neurology,Hemet
4,1285716514,A,Muir,Dentist,San Diego


In [5]:
sunshine.head()

,id,fname,lname,type,city
0,746174,ENRICA,FUNG,Allopathic & Osteopathic Physicians|Internal M...,LOMA LINDA
1,652594,Richard,Young,Dental Providers|Dentist|General Practice,SAN BERNARDINO
2,370086,HEATHER,HINSHELWOOD,Allopathic & Osteopathic Physicians|Emergency ...,29 PALMS
3,639276,VICTORIA,ZHENG,Allopathic & Osteopathic Physicians|Anesthesio...,TORRANCE
4,426528,ODOCHI,NWAGWU,Allopathic & Osteopathic Physicians|Pediatrics,ADELANTO


In [6]:
def processType(x):
    s = x.split('|')
    return s[-1]

In [7]:
def processData(df):
    cols = df.columns.to_list()
    for col in cols:
        df[col] = df[col].astype(str)
        df[col] = df[col].apply(lambda x: x.lower().strip())
    df['type'] = df['type'].apply(lambda x: processType(x))
    df['combined'] = df['fname'] + ' ' + df['lname'] + ' ' + df['city'] + ' ' + df['type']
    return df

In [8]:
sunshine = processData(sunshine)
sunshine.head()

,id,fname,lname,type,city,combined
0,746174,enrica,fung,internal medicine,loma linda,enrica fung loma linda internal medicine
1,652594,richard,young,general practice,san bernardino,richard young san bernardino general practice
2,370086,heather,hinshelwood,emergency medicine,29 palms,heather hinshelwood 29 palms emergency medicine
3,639276,victoria,zheng,anesthesiology,torrance,victoria zheng torrance anesthesiology
4,426528,odochi,nwagwu,pediatrics,adelanto,odochi nwagwu adelanto pediatrics


In [9]:
part_d = processData(part_d)
part_d.head()

,id,fname,lname,type,city,combined
0,1447344023,aleksandr,itkin,dermatology,san diego,aleksandr itkin san diego dermatology
1,1184722209,david,hughes,dentist,milpitas,david hughes milpitas dentist
2,1164561353,pamela,knickerbocker,family practice,shasta lake,pamela knickerbocker shasta lake family practice
3,1225025422,a,jaffer,neurology,hemet,a jaffer hemet neurology
4,1285716514,a,muir,dentist,san diego,a muir san diego dentist


Here we will do a naive matching. Based on just the 'combined' column. If there is an exact match we will accept this.

In [13]:
merged_part_d = part_d.merge(right=sunshine,on='combined',how='left')
merged_part_d.head()

,id_x,fname_x,lname_x,type_x,city_x,combined,id_y,fname_y,lname_y,type_y,city_y
0,1447344023,aleksandr,itkin,dermatology,san diego,aleksandr itkin san diego dermatology,292163,aleksandr,itkin,dermatology,san diego
1,1184722209,david,hughes,dentist,milpitas,david hughes milpitas dentist,NaN,NaN,NaN,NaN,NaN
2,1164561353,pamela,knickerbocker,family practice,shasta lake,pamela knickerbocker shasta lake family practice,NaN,NaN,NaN,NaN,NaN
3,1225025422,a,jaffer,neurology,hemet,a jaffer hemet neurology,NaN,NaN,NaN,NaN,NaN
4,1285716514,a,muir,dentist,san diego,a muir san diego dentist,NaN,NaN,NaN,NaN,NaN


In [15]:
merged_sunshine = sunshine.merge(right=part_d,on='combined',how='left')
merged_sunshine.head()

,id_x,fname_x,lname_x,type_x,city_x,combined,id_y,fname_y,lname_y,type_y,city_y
0,746174,enrica,fung,internal medicine,loma linda,enrica fung loma linda internal medicine,NaN,NaN,NaN,NaN,NaN
1,652594,richard,young,general practice,san bernardino,richard young san bernardino general practice,NaN,NaN,NaN,NaN,NaN
2,370086,heather,hinshelwood,emergency medicine,29 palms,heather hinshelwood 29 palms emergency medicine,1235289984,heather,hinshelwood,emergency medicine,29 palms
3,639276,victoria,zheng,anesthesiology,torrance,victoria zheng torrance anesthesiology,NaN,NaN,NaN,NaN,NaN
4,426528,odochi,nwagwu,pediatrics,adelanto,odochi nwagwu adelanto pediatrics,NaN,NaN,NaN,NaN,NaN


In [ ]:
merged_part_d_no_matches = merged_part_d[merged_part_d.id_y.isnull()]
merged_sunshine_no_matches = merged_sunshine[merged_sunshine.id_y.isnull()]

In [17]:
# merged_part_d_no_matches.to_csv('../Data/Outputs_Cleanup/part_d_no_matches.csv',index=False)
# merged_sunshine_no_matches.to_csv('../Data/Outputs_Cleanup/sunshine_no_matches.csv',index=False)

In [10]:
# sunshine.to_csv('../Data/Outputs_Cleanup/sunshine_comb.csv',index=False)
# part_d.to_csv('../Data/Outputs_Cleanup/part_d_comb.csv',index=False)